In [16]:
!pip install pykeops[full] > log.log


# Nystrom implementation

In [96]:
import numpy as np
import torch
from sklearn.utils import check_random_state, as_float_array
from scipy.linalg import svd
from pykeops.torch import LazyTensor
from sklearn.kernel_approximation import Nystroem
import scipy
from pykeops.numpy import LazyTensor
import pykeops.config
from pykeops.numpy.cluster import grid_cluster
from pykeops.numpy.cluster import cluster_ranges_centroids
from pykeops.numpy.cluster import sort_clusters
from pykeops.numpy.cluster import from_matrix

dtype = "float32"

In [115]:
##############################################################################

class LazyNystrom_N:
    '''
        Class to implement Nystrom on torch LazyTensors.
        This class works as an interface between lazy tensors and 
        the Nystrom algorithm in NumPy.

        * The fit method computes K^{-1}_q.

        * The transform method maps the data into the feature space underlying
        the Nystrom-approximated kernel.

        * The method K_approx directly computes the Nystrom approximation.

        Parameters:

        n_components [int] = how many samples to select from data.
        kernel [str] = type of kernel to use. Current options = {linear, rbf}.
        gamma [float] = exponential constant for the RBF kernel. 

    '''
  
    def __init__(self, n_components=500, kernel='linear', gamma:float = 1., 
                 random_state=None): 

        self.n_components = n_components
        self.kernel = kernel
        self.random_state = random_state
        self.gamma = gamma


    def fit(self, X:LazyTensor):
        ''' 
        Args:   X = lazy tensor with features of shape 
                (1, n_samples, n_features)

        Returns: Fitted instance of the class
        '''

        # Basic checks
        assert type(X) == LazyTensor, 'Input to fit(.) must be a LazyTensor.'
        assert X.shape[1] >= self.n_components, f'The application needs X.shape[1] >= n_components.'

        X = X.sum(dim=0).numpy()
        # Number of samples
        n_samples = X.shape[0]
        # Define basis
        rnd = check_random_state(self.random_state)
        inds = rnd.permutation(n_samples)
        basis_inds = inds[:self.n_components]
        basis = X[basis_inds]
        # Build smaller kernel
        basis_kernel = self._pairwise_kernels(basis, kernel = self.kernel)  
        # Get SVD
        U, S, V = svd(basis_kernel)
        S = np.maximum(S, 1e-12)
        self.normalization_ = np.dot(U / np.sqrt(S), V)
        self.components_ = basis
        self.component_indices_ = inds
        return self


    def _pairwise_kernels(self, x:np.array, y:np.array = None, kernel='linear',
                          gamma = 1., mask=False):
        '''Helper function to build kernel
        
        Args:   X = torch tensor of dimension 2.
                K_type = type of Kernel to return
        '''
        
        if y is None:
            y = x
        if kernel == 'linear':
            K = x @ y.T 
        elif kernel == 'rbf':
            K =  ( (x[:,None,:] - y[None,:,:])**2 ).sum(-1)
            K = np.exp(- gamma* K)

        if mask:
            ranges_ij = binary_mask(x,y)
            K.ranges = ranges_ij  # block-sparsity pattern
  
        return K

    

    def transform(self, X:LazyTensor) -> LazyTensor:
        ''' Applies transform on the data.
        
        Args:
            X [LazyTensor] = data to transform
        Returns
            X [LazyTensor] = data after transformation
        '''
        
        X = X.sum(dim=0)
        K_nq = self._pairwise_kernels(X, self.components_, self.kernel)

        return LazyTensor((K_nq @ self.normalization_.T)[None,:,:])


    def binary_mask(x, y):
        eps = 0.05  # Size of our square bins
        #perform clustering
        x_labels = grid_cluster(x, eps)  # class labels
        y_labels = grid_cluster(y, eps)  # class labels
        #centroids and memory footpring of each class
        self.x_ranges, self.x_centroids, _ = cluster_ranges_centroids(x, x_labels)
        self.y_ranges, self.y_centroids, _ = cluster_ranges_centroids(y, y_labels)
        #sort so that all clusters are stored contiguously in memory:
        x, self.x_labels = sort_clusters(x, x_labels)
        y, self.y_labels = sort_clusters(y, y_labels)
        # binary mask
        sigma = 0.05  # Characteristic length of interaction
        # Compute a coarse Boolean mask:
        D = np.sum((self.x_centroids[:, None, :] - self.y_centroids[None, :, :]) ** 2, 2)
        self.keep = D < (4 * sigma) ** 2
        ranges_ij = from_matrix(self.x_ranges, self.y_ranges, self.keep)
        return ranges_ij

    def plotting(ranges_ij ):
        # Find the cluster centroid which is closest to the (.43,.6) point:
        dist_target = np.sum(((self.x_centroids - np.array([0.43, 0.6]).astype(dtype)) ** 2), axis=1)
        clust_i = np.argmin(dist_target)

        if M + N <= 500000:
            ranges_i, slices_j, redranges_j = self.ranges_ij[0:3]
            start_i, end_i = ranges_i[clust_i]  # Indices of the points that make up our cluster
            start, end = (
                slices_j[clust_i - 1],
                slices_j[clust_i],
            )  # Ranges of the cluster's neighbors

            keep = self.keep.astype(float)
            keep[clust_i] += 2

            plt.ion()
            plt.matshow(keep)

            plt.figure(figsize=(10, 10))

            plt.scatter(
                x[:, 0],
                x[:, 1],
                c=self.x_labels,
                cmap=plt.cm.Wistia,
                s=25 * 500 / len(x),
                label="Target points",
            )
            plt.scatter(
                y[:, 0],
                y[:, 1],
                c=self.y_labels,
                cmap=plt.cm.winter,
                s=25 * 500 / len(y),
                label="Source points",
            )

            # Target clusters:
            for start_j, end_j in redranges_j[start:end]:
                plt.scatter(
                    y[start_j:end_j, 0], y[start_j:end_j, 1], c="magenta", s=50 * 500 / len(y)
                )

            # Source cluster:
            plt.scatter(
                x[start_i:end_i, 0],
                x[start_i:end_i, 1],
                c="cyan",
                s=10,
                label="Cluster {}".format(clust_i),
            )

            plt.scatter(
                self.x_centroids[:, 0],
                self.x_centroids[:, 1],
                c="black",
                s=10,
                alpha=0.5,
                label="Cluster centroids",
            )

            plt.legend(loc="lower right")

            # sphinx_gallery_thumbnail_number = 2
            plt.axis("equal")
            plt.axis([0, 1, 0, 1])
            plt.tight_layout()
            plt.show(block=True)
                

    
    def K_approx(self, X:LazyTensor) -> LazyTensor:
        ''' Function to return Nystrom approximation to the kernel.
        
        Args:
            X[LazyTensor] = data used in fit(.) function.
        Returns
            K[LazyTensor] = Nystrom approximation to kernel'''
        
        X = X.sum(dim=0).numpy()
        K_nq = self._pairwise_kernels(X, self.components_, self.kernel)
        K_approx = K_nq @ self.normalization_ @ K_nq.T
        K_approx = torch.tensor(K_approx)
        return LazyTensor(K_approx[None,:,:])





In [116]:
# NUMPY gen data
M, N = (5000, 5000) if pykeops.config.gpu_available else (2000, 2000)

t = np.linspace(0, 2 * np.pi, M + 1)[:-1]
x = np.stack((0.4 + 0.4 * (t / 7) * np.cos(t), 0.5 + 0.3 * np.sin(t)), 1)
x = x + 0.01 * np.random.randn(*x.shape)
x = x.astype(dtype)

y = np.random.randn(N, 2).astype(dtype)
y = y / 10 + np.array([0.6, 0.6]).astype(dtype)


In [107]:
nystroem = LazyNystrom_N(kernel='rbf')
X = LazyTensor(x[:, None, :])
data = nystroem.fit(X)
transformed_data = nystroem.transform(X)#.sum(dim=0)

print("transformed_data shape")
print(transformed_data.shape)
transformed1 = transformed_data.sum(dim=0)
transformed2 = transformed_data.sum(dim=1)
print(transformed1.shape, transformed2.shape)

<class 'pykeops.numpy.lazytensor.LazyTensor.LazyTensor'>
<class 'numpy.ndarray'>
kernel K_nq and normalisation
[[0.97277653 0.94454575 0.9583175  ... 0.93694556 0.91865563 0.91584736]
 [0.9710361  0.9485484  0.96028477 ... 0.93645626 0.9224755  0.9138108 ]
 [0.9725648  0.9498752  0.9640769  ... 0.9405182  0.9211253  0.91736037]
 ...
 [0.875665   0.7318018  0.75065804 ... 0.7782721  0.7497433  0.8007208 ]
 [0.88536566 0.72463804 0.75535893 ... 0.7910082  0.73532104 0.81723213]
 [0.88104385 0.7367087  0.75702375 ... 0.78487855 0.75305444 0.8070697 ]]
[[-98.4149      -2.7437384  -11.836015   ...   2.3420856    1.599436
    0.42536885]
 [  7.4600897    6.114054    25.0116     ...   7.319084    -3.858205
   -9.145769  ]
 [109.40387     21.012632    -7.9959846  ...  -2.5300145    1.2492018
  -11.485975  ]
 ...
 [-77.89973      6.2144027    0.82074106 ...   4.882506     7.2108083
  -13.289615  ]
 [ 78.70853      4.4706807    4.6666555  ...   8.346193   -16.480232
   12.601715  ]
 [-15.062228 

In [108]:
# Find the cluster centroid which is closest to the (.43,.6) point:
dist_target = np.sum(((x_centroids - np.array([0.43, 0.6]).astype(dtype)) ** 2), axis=1)
clust_i = np.argmin(dist_target)

if M + N <= 500000:
    ranges_i, slices_j, redranges_j = ranges_ij[0:3]
    start_i, end_i = ranges_i[clust_i]  # Indices of the points that make up our cluster
    start, end = (
        slices_j[clust_i - 1],
        slices_j[clust_i],
    )  # Ranges of the cluster's neighbors

    keep = keep.astype(float)
    keep[clust_i] += 2

    plt.ion()
    plt.matshow(keep)

    plt.figure(figsize=(10, 10))

    plt.scatter(
        x[:, 0],
        x[:, 1],
        c=x_labels,
        cmap=plt.cm.Wistia,
        s=25 * 500 / len(x),
        label="Target points",
    )
    plt.scatter(
        y[:, 0],
        y[:, 1],
        c=y_labels,
        cmap=plt.cm.winter,
        s=25 * 500 / len(y),
        label="Source points",
    )

    # Target clusters:
    for start_j, end_j in redranges_j[start:end]:
        plt.scatter(
            y[start_j:end_j, 0], y[start_j:end_j, 1], c="magenta", s=50 * 500 / len(y)
        )

    # Source cluster:
    plt.scatter(
        x[start_i:end_i, 0],
        x[start_i:end_i, 1],
        c="cyan",
        s=10,
        label="Cluster {}".format(clust_i),
    )

    plt.scatter(
        x_centroids[:, 0],
        x_centroids[:, 1],
        c="black",
        s=10,
        alpha=0.5,
        label="Cluster centroids",
    )

    plt.legend(loc="lower right")

    # sphinx_gallery_thumbnail_number = 2
    plt.axis("equal")
    plt.axis([0, 1, 0, 1])
    plt.tight_layout()
    plt.show(block=True)

NameError: ignored

In [ ]:

##########################################################################

# Same as LazyNystrom_N but written with Pytorch

class LazyNystrom_T:
    '''
        Class to implement Nystrom on torch LazyTensors.
        This class works as an interface between lazy tensors and 
        the Nystrom algorithm in NumPy.

        * The fit method computes K^{-1}_q.

        * The transform method maps the data into the feature space underlying
        the Nystrom-approximated kernel.

        * The method K_approx directly computes the Nystrom approximation.

        Parameters:

        n_components [int] = how many samples to select from data.
        kernel [str] = type of kernel to use. Current options = {linear, rbf}.
        gamma [float] = exponential constant for the RBF kernel. 

    '''
  
    def __init__(self, n_components=100, kernel='linear',  gamma:float = 1., 
                 random_state=None ):
        
        self.n_components = n_components
        self.kernel = kernel
        self.random_state = random_state
        self.gamma = gamma


    def fit(self, X:LazyTensor):
        ''' 
        Args:   X = torch lazy tensor with features of shape # WHY INPUT IS LAZY TENSOR
                (1, n_samples, n_features)

        Returns: Fitted instance of the class
        '''

        # Basic checks: we have a lazy tensor and n_components isn't too large
        assert type(X) == LazyTensor, 'Input to fit(.) must be a LazyTensor.'
        assert X.shape[0] >= self.n_components, f'The application needs X.shape[1] >= n_components.'

        X = X#.sum(dim=0) # ASK ABOUT REDOCTION
        
        # Number of samples
        n_samples = X.size(0)
        # Define basis
        rnd = check_random_state(self.random_state)
        inds = rnd.permutation(n_samples)
        basis_inds = inds[:self.n_components]
        basis = X[basis_inds]
        # Build smaller kernel
        basis_kernel = self._pairwise_kernels(basis, kernel = self.kernel)  
        # Get SVD
        U, S, V = torch.svd(basis_kernel)
        S = torch.maximum(S, torch.ones(S.size()) * 1e-12)
        self.normalization_ = torch.mm(U / np.sqrt(S), V.t())
        self.components_ = basis
        self.component_indices_ = inds
        
        return self


    def _pairwise_kernels(self, x:torch.tensor, y:torch.tensor = None, kernel='linear',
                          gamma = 1.) -> torch.tensor:
        '''Helper function to build kernel
        
        Args:   X = torch tensor of dimension 2.
                K_type = type of Kernel to return
        '''
        
        if y is None:
            y = x
        if kernel == 'linear':
            K = x @ y.T
        elif kernel == 'rbf':
            K =  ( (x[:,None,:] - y[None,:,:])**2 ).sum(-1)
            K = torch.exp(- gamma * K )

        return K

    def transform(self, X:LazyTensor) -> LazyTensor:
        ''' Applies transform on the data.
        
        Args:
            X [LazyTensor] = data to transform
        Returns
            X [LazyTensor] = data after transformation
        '''
        
        X = X.sum(dim=0)
        K_nq = self._pairwise_kernels(X, self.components_, self.kernel)
        return LazyTensor((K_nq @ self.normalization_.t())[None,:,:])

    
    def K_approx(self, X:LazyTensor) -> LazyTensor:
        ''' Function to return Nystrom approximation to the kernel.
        
        Args:
            X[LazyTensor] = data used in fit(.) function.
        Returns
            K[LazyTensor] = Nystrom approximation to kernel'''
        
        X = X.sum(dim=0)
        K_nq = self._pairwise_kernels(X, self.components_, self.kernel)
        K_approx = K_nq @ self.normalization_ @ K_nq.t()
        return LazyTensor(K_approx[None,:,:])
